In [1]:
import os
import csv
import cv2
from skimage.feature import hog
from sklearn.externals import joblib

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
    return images 

def load_sets():
    sa_knjigom = load_images_from_folder("skup_podataka/saKnjigom")
    bez_knjige = load_images_from_folder("skup_podataka/bezKnjiga")
    test_skup = load_images_from_folder("skup_podataka/test")
    return sa_knjigom, bez_knjige, test_skup

In [3]:
with_books, without_books, test_set = load_sets()
print("Slike sa knjigama:", len(with_books))
print("Slike bez knjiga:", len(without_books))
print("Slike za test:", len(test_set))

lista_svih = []
labele = []
print("- pozz knjige")
br = 1
for img in with_books:
    features = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2")
    lista_svih.append(features)
    labele.append(1)
    if br % 100 == 0 or br > 208:
        print("--", br, " ", len(features))
    br += 1
    
print("- neg knjige")
br = 1
for img in without_books:
    features = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2")
    lista_svih.append(features)
    labele.append(0)
    if br % 100 == 0 or br > 208:
        print("--", br, " ", features)
    br += 1
print("- kraaj knjiga")

clf = joblib.load("pedestrian.pkl")
print(clf)
clf.fit(lista_svih, labele)

Slike sa knjigama: 210
Slike bez knjiga: 210
Slike za test: 150
- pozz knjige
-- 100   1425636
-- 200   1425636
-- 209   1425636
-- 210   1425636
- neg knjige
-- 100   [0.12048013 0.00468876 0.02853342 ... 0.01930906 0.02878426 0.        ]
-- 200   [0.17440152 0.         0.         ... 0.         0.         0.        ]
-- 209   [0.42871029 0.         0.01732251 ... 0.27041089 0.23306476 0.26117683]
-- 210   [0.30927826 0.08554968 0.07624716 ... 0.06607455 0.14553686 0.12925364]
- kraaj knjiga
LinearSVC(C=0.01, class_weight='balanced', verbose=1)


D:\soft_vezbe\sc-2021-siit\env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[LibLinear]

LinearSVC(C=0.01, class_weight='balanced', verbose=1)

In [4]:
#print(clf)
resenje = []
with open('skup_podataka/test_podaci.csv', encoding="utf8", newline='\n') as f:
    lines = csv.reader(f, delimiter=',')
    for row in lines:
        if (row[0] == "br_knjiga"):
            continue
        resenje.append(row[0] != 0)

br = 0
br_tacnih = 0
for img in test_set:
    features = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2")
    listica = [features, features]
    rez = clf.predict(listica)
    if (rez[0] == resenje[br]):
        br_tacnih += 1
    br += 1

print ("Ukupno testiranih:", br)
print("Broj tacnih:", br_tacnih)                       # crno-bela slika 94
print("Procentualno:", (br_tacnih/br)*100, "%")        # crno-bela slika 63%

Ukupno testiranih: 150
Broj tacnih: 103
Procentualno: 68.66666666666667 %
